# Anexo 3

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from ASHRAE_tools import meter_dict
from ASHRAE_tools import submit
import ASHRAE_data_prep

meter_dict = meter_dict()

## Modelo de Regresión 1

In [2]:
X_train, y_train = ASHRAE_data_prep.train_reg()

In [6]:
X_train.head()

,building_id,meter,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,wday,week
0,0,0,0,7432,25.0,20.0,1019.7,0.0,0.0,0,4,53
1,0,0,0,7432,24.4,21.1,1020.2,70.0,1.5,1,4,53
2,0,0,0,7432,22.8,21.1,1020.2,0.0,0.0,2,4,53
3,0,0,0,7432,21.1,20.6,1020.1,0.0,0.0,3,4,53
4,0,0,0,7432,20.0,20.0,1020.0,250.0,2.6,4,4,53


In [7]:
mod_reg = LinearRegression()

In [8]:
mod_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
del X_train

In [10]:
mod_reg.coef_

array([[-2.65885923e+01,  2.08145493e+03,  2.34789920e+03,
         2.92220651e-02, -1.17494723e+02,  1.39814835e+02,
         2.28839207e+00, -3.57551059e+00,  5.70947189e+02,
         1.05215779e+01, -5.52471680e+01, -7.39483025e+01]])

In [11]:
row, X_test = ASHRAE_data_prep.test_reg()

In [12]:
X_test.shape

(41697600, 12)

In [13]:
y = mod_reg.predict(X_test)

In [54]:
submit(row)

### Cross validation

In [18]:
X_train, y_train = ASHRAE_data_prep.train_reg()
mod_reg = LinearRegression()

In [8]:
kfold = GroupKFold(n_splits = 3)

In [9]:
fold_metrics = []

In [28]:
for train_index, test_index in kfold.split(X_train, y_train, X_train['building_id']):
    xv_train, xv_test = X_train.iloc[train_index], X_train.iloc[test_index]
    yv_train, yv_test = y_train.iloc[train_index], y_train.iloc[test_index]
    mod_reg.fit(xv_train, yv_train)
    y_pred = np.maximum(mod_reg.predict(xv_test),0)
    metric = np.sqrt(mean_squared_log_error(yv_test,y_pred))
    fold_metrics.append(metric)

In [29]:
fold_metrics

[4.358489391356827, 2.8934359319885976, 4.40673941672632]

## Modelo de Regresión 2

In [2]:
X_train, y_train = ASHRAE_data_prep.train_reg_cat()

In [3]:
X_train.shape

(20216100, 29)

In [3]:
X_train.head()

,building_id,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,wday,week,meter_1,meter_2,meter_3
0,0,7432,25.0,20.0,0.0,1019.7,0.0,0.0,0,4,53,0.0,0.0,0.0
1,0,7432,24.4,21.1,-1.0,1020.2,70.0,1.5,1,4,53,0.0,0.0,0.0
2,0,7432,22.8,21.1,0.0,1020.2,0.0,0.0,2,4,53,0.0,0.0,0.0
3,0,7432,21.1,20.6,0.0,1020.1,0.0,0.0,3,4,53,0.0,0.0,0.0
4,0,7432,20.0,20.0,-1.0,1020.0,250.0,2.6,4,4,53,0.0,0.0,0.0


### Cross validation

In [4]:
mod_reg = LinearRegression()

In [5]:
kfold = GroupKFold(n_splits = 2)

In [6]:
fold_metrics = []

In [7]:
for train_index, test_index in kfold.split(X_train, y_train, X_train['building_id']):
    xv_train, xv_test = X_train.iloc[train_index], X_train.iloc[test_index]
    yv_train, yv_test = y_train.iloc[train_index], y_train.iloc[test_index]
    mod_reg.fit(xv_train, yv_train)
    y_pred = np.maximum(mod_reg.predict(xv_test),0)
    metric = np.sqrt(mean_squared_log_error(yv_test,y_pred))
    fold_metrics.append(metric)

In [8]:
fold_metrics

[3.0229925026832896, 4.318974829424748]

### Modelo completo

In [9]:
mod_reg_cat = LinearRegression()
mod_reg_cat.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
X_test = ASHRAE_data_prep.test_reg_cat()

In [13]:
X_test.head()

,building_id,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,wday,week,meter_1,meter_2,meter_3
row_id,,,,,,,,,,,,,,
0,0,7432,17.8,11.7,0.0,1021.4,100.0,3.6,0,6,52,0.0,0.0,0.0
129,0,7432,17.8,12.8,0.0,1022.0,130.0,3.1,1,6,52,0.0,0.0,0.0
258,0,7432,16.1,12.8,0.0,1021.9,140.0,3.1,2,6,52,0.0,0.0,0.0
387,0,7432,17.2,13.3,0.0,1022.2,140.0,3.1,3,6,52,0.0,0.0,0.0
516,0,7432,16.7,13.3,0.0,1022.3,130.0,2.6,4,6,52,0.0,0.0,0.0


In [14]:
y_pred = mod_reg_cat.predict(X_test)

In [47]:
sub = pd.DataFrame(np.maximum(0,y_pred), index = X_test.index, columns = ['meter_reading'])

In [55]:
sub.sort_values(by = 'row_id', inplace = True)

In [57]:
sub.shape

(41697600, 1)

In [58]:
sub.to_csv('./submission.csv')

## Modelo de Regresión Lasso

In [3]:
train = ASHRAE_data_prep.train_lasso()

In [3]:
train.head()

,building_id,meter_reading,air_temperature,hour,wday,week,1,2,3,4,...,1442,1443,1444,1445,1446,1447,1448,meter_1,meter_2,meter_3
0,0,0.0,5.6,6,0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,4.4,12,0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,17.2,17,0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,20.6,22,0,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,11.1,6,1,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train.shape

(1542162, 1457)

In [4]:
y_train = train['meter_reading']

In [5]:
X_train = train

In [6]:
X_train.drop('meter_reading', inplace = True, axis = 1)

In [10]:
X_train.head()

,building_id,air_temperature,hour,wday,week,1,2,3,4,5,...,1442,1443,1444,1445,1446,1447,1448,meter_1,meter_2,meter_3
0,0,5.6,6,0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,4.4,12,0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,17.2,17,0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,20.6,22,0,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,11.1,6,1,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
mod_lasso = Lasso()

In [9]:
mod_lasso.fit(X_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [15]:
mod_lasso.coef_

array([-1.36714972e+00,  3.83996163e+01,  1.27601803e+00, ...,
        2.70459142e+03,  1.13191451e+04,  1.37954578e+03])

In [ ]:
y_pred = mod_lasso.predict(X_test)